In [6]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# CHANGE ME!
TASK_ID = 'unique_beer_drinkers'
AUTHOR = 'will'

In [8]:
task = Task(
    id=TASK_ID,
    category="First-order logic",
    name="People who like a unique set of beer",
    description="Find all people whose preferred set of beers is distinct from each other person's preferred set",
    plan=[
        Plan(id="collect", description="Get all beers that a person likes"),
        Plan(id="iter", description="Iterate over all other people in the dataset"),
        Plan(id="compare", description="Compare two beer sets for equality")
    ],
    sample_input={
        "likes": [
            {"name": "will", "beer": "ipa"},
            {"name": "will", "beer": "lager"},
            {"name": "scott", "beer": "ipa"},
            {"name": "scott", "beer": "stout"},
            {"name": "gleb", "beer": "ipa"},
            {"name": "gleb", "beer": "stout"},
            {"name": "fred", "beer": "ipa"},
            {"name": "fred", "beer": "lager"},
            {"name": "fred", "beer": "stout"},
        ]
    },
    sample_output=["will", "fred"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [9]:
sql = replace(
    prototype,
    language='sql',
    source='''SELECT DISTINCT L1.name
FROM likes L1
WHERE NOT EXISTS(
    SELECT *
    FROM likes L2
    WHERE L1.name != L2.name
    AND NOT EXISTS(
        SELECT *
        FROM likes L3
        WHERE L3.name = L2.name
        AND NOT EXISTS(
            SELECT *
            FROM likes L4
            WHERE L4.name = L1.name
            AND L4.beer = L3.beer))
    AND NOT EXISTS(
        SELECT *
        FROM likes L5
        WHERE L5.name = L1.name
        AND NOT EXISTS(
            SELECT *
            FROM likes L6
            WHERE L6.name = L2.name
            AND L6.beer= L5.beer)))''')
sql.execute(task)
sql.save()

In [10]:
datalog1 = replace(
    prototype,
    language='datalog',
    implementation='count',
    source='''unique_beer_drinkers(Name) :-
  likes(_, Name),
  0 = count : {
    likes(_, Other),
    Other != Name,
    Shared_count = count : { likes(Beer, Other), likes(Beer, Name) },
    count : likes(_, Other) = Shared_count,
    count : likes(_, Name) = Shared_count
  }.''')
datalog1.execute(task)
datalog1.save()

In [11]:
datalog2 = replace(
    prototype,
    language='datalog',
    implementation='neg_exist',
    source='''.decl differ(a:symbol, b:symbol)
differ(A, B) :- likes(Beer, A), likes(_, B), !likes(Beer, B).
differ(A, B) :- likes(_, A), likes(Beer, B), !likes(Beer, A).

.decl exists_same(a:symbol)
exists_same(Name) :- likes(_, Other), likes(_, Name), Name != Other, !differ(Name, Other).

unique_beer_drinkers(Name) :- likes(_, Name), !exists_same(Name).''')
datalog2.execute(task)
datalog2.save()

In [12]:
pandas = replace(
    prototype,
    language='python-pandas',
    author='will',
    source='''def unique_beer_drinkers(likes):
  likes_per_person = (likes
    .groupby('name')
    .beer.unique().map(set)
    .reset_index())

  def check_not_exists(row):
      other_people = likes_per_person[likes_per_person.name != row['name']]
      return not (other_people.beer == row['beer']).any()
  
  unique_drinkers = likes_per_person[likes_per_person.apply(check_not_exists, axis=1)]
  return unique_drinkers.name.tolist()''')
pandas.execute(task)
pandas.save()

In [13]:
python_functional = replace(
    prototype,
    language='python-functional',
    source='''def unique_beer_drinkers(likes):
  people = set([row['name'] for row in likes])
  likes_per_person = {
    name: set([row['beer'] for row in likes if row['name'] == name])
    for name in people
  }
    
  return [
      name
      for name, beers in likes_per_person.items()
      if not any([
          other_name != name and beers == other_beers
          for other_name, other_beers in likes_per_person.items()
      ])
  ]''')
python_functional.execute(task)
python_functional.save()

In [14]:
python_imperative = replace(
    prototype,
    language='python-imperative',
    source='''def unique_beer_drinkers(likes):
  likes_per_person = defaultdict(set)
  for row in likes:
    likes_per_person[row['name']].add(row['beer'])
    
  unique = []
  for p1, p1_likes in likes_per_person.items():
    is_unique = True
    for p2, p2_likes in likes_per_person.items():
      if p1 == p2:
        continue
        
      if p1_likes == p2_likes:
        is_unique = False
        break
    if is_unique:
      unique.append(p1)
      
  return unique''')
python_imperative.execute(task)
python_imperative.save()